In [92]:
import pandas as pd
import funzioni as fx
import numpy as np
import yfinance as yf
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score

lista_tickers = pd.read_parquet("lista_ticker.parquet")
lista_tickers = lista_tickers.sample(frac=1)

def test_trading():
    bilancio = BILANCIO_INIZIALE
    pos_aperta = False
    tp = sl = n_azioni = prezzo_acquisto = prezzo_tot = 0
    giorni_posizione = 0
    posizioni = pd.DataFrame(columns=["Bilancio", "Direzione", "Prezzo_un", "n_azioni", "Prezzo_tot", "TP", "SL", "Open", "High", "Low", "Esito", "P_L"], index=df_X.index)

    for idx, row in df_X.iterrows():
        if bilancio <= 0:
            break
        print(f"\r{idx} Bilancio = {bilancio}                ", end=" ", flush=True)
        if pos_aperta == False:
            if row["prob_True"] > PROBABILITA_PER_ACQUISTO and bilancio > 0:
                #COMPRA
                prezzo_acquisto = row["Open"]
                n_azioni = bilancio // prezzo_acquisto
                prezzo_tot = n_azioni * prezzo_acquisto  
                bilancio -= prezzo_tot          
                tp = prezzo_acquisto * (1 + TP)
                sl = prezzo_acquisto * (1 - SL)
                pos_aperta = True
                giorni_posizione = 0
                posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "COMPRA", "Prezzo_un": prezzo_acquisto, "n_azioni": n_azioni, "Prezzo_tot": -prezzo_tot, "TP": tp, "SL": sl, "Open": row["Open"], "High": row["High"], "Low": row["Low"],}
        else:
            if giorni_posizione == 20:
                prezzo_tot_vendita = row['Close'] * n_azioni
                bilancio += prezzo_tot_vendita
                pos_aperta = False
                if prezzo_tot_vendita > prezzo_tot:
                    esito = 'VINCITA'
                else:
                    esito = 'PERDITA'
                pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita),0)
                posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "VENDI", "Prezzo_un": row['Close'], "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": esito, "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
            if row["High"] >= tp:
                prezzo_tot_vendita = tp * n_azioni
                bilancio += prezzo_tot_vendita
                pos_aperta = False
                giorni_posizione = 0
                pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita),0)
                posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "VENDI", "Prezzo_un": tp, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "VINCITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
            elif row["Low"] <= sl:
                prezzo_tot_vendita = sl * n_azioni
                bilancio += prezzo_tot_vendita
                pos_aperta = False
                giorni_posizione = 0
                pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita),0)
                posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "VENDI", "Prezzo_un": sl, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "PERDITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
            else:
                giorni_posizione += 1

    if pos_aperta: 
        bilancio += prezzo_tot
        print(f"\r{idx} Bilancio = {bilancio}                ", end=" ", flush=True)
    posizioni.to_excel("posizioni.xlsx")
    #df_X.to_excel("X.xlsx")
    print(f'\nVincite: {posizioni.loc[posizioni["Esito"] == "VINCITA", "Esito"].count()}, vincita max: {posizioni.loc[posizioni["Esito"] == "VINCITA", "P_L"].max()} %')
    print(f'Perdite: {posizioni.loc[posizioni["Esito"] == "PERDITA", "Esito"].count()}, perdita max: {posizioni.loc[posizioni["Esito"] == "PERDITA", "P_L"].min()} %')



In [93]:
TP = 1
SL = 0.01
PROBABILITA_PER_ACQUISTO = 0.5
BILANCIO_INIZIALE = 1000
SIMBOLO_TEST = lista_tickers["Ticker"].iloc[0]

In [94]:
print("\033[42m" + SIMBOLO_TEST + "\033[0m")
ticker = yf.download(SIMBOLO_TEST, start='2010-11-01', end='2023-12-31', progress=True)
if ticker["Close"].iloc[-1] >= 1:
    ticker.index = ticker.index.date
    print("Calcolo indicatori ticker", flush=True)
    ticker = fx.crea_indicatori(ticker)
    ticker = fx.imposta_target(ticker)
    ticker.dropna(axis=0, inplace=True)

    print("Definizione features e target", flush=True)
    idx, X, Y, scalers_prezzo = fx.to_XY(ticker, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, bilanciamento=0)
print('Previsione')
f1_score = F1Score(num_classes=1, average='macro', threshold=PROBABILITA_PER_ACQUISTO)
model = load_model("model.h5", custom_objects={'Addons>F1Score': f1_score})
y_pred = model.predict(X)
result = model.evaluate(X, Y)
print(f"ticker.shape:{ticker.shape}")
print(f"X.shape:{X.shape}")
print(f"Y.shape:{Y.shape}")
print(f"y_pred.shape:{y_pred.shape}")
print(f'idx:{len(idx)}')
X = X[:, 59, :len(fx.features_prezzo)]

df_X = ticker.loc[ticker.index.intersection(idx)]
y_pred = y_pred.reshape(-1,)
df_y_pred = pd.DataFrame(y_pred, columns=['prob_True'], index=idx)
df_X = pd.concat([df_X, df_y_pred], axis=1)
test_trading()

SKYW
[*********************100%%**********************]  1 of 1 completed
Calcolo indicatori ticker
Definizione features e target
Previsione
95/95 [==============================] - 1s 6ms/step - loss: 0.4786 - f1_score: 0.0000e+00 - precision: 0.8599 - recall: 0.1684
ticker.shape:(3157, 53)
X.shape:(3038, 120, 10)
Y.shape:(3038, 1)
y_pred.shape:(3038, 1)
idx:3038
2023-10-09 Bilancio = 3278.453871612548                    
Vincite: 10, vincita max: 28.0 %
Perdite: 34, perdita max: -1.0 %


In [ ]:
class Trade:
    def __init__(self, nome_ticker, data_inizio, data_fine, importo_da_investire, probabilità_per_acquisto, stop_loss, take_profit, giorni_max_posizione) -> None:
        self.SIMBOLO = nome_ticker
        self.DATA_INIZIO = data_inizio
        self.DATA_FINE = data_fine
        self.BILANCIO_INIZIALE = importo_da_investire
        self.PROBABILITA_PER_ACQUISTO = probabilità_per_acquisto
        self.SL = stop_loss
        self.TP = take_profit
        self.GIORNI_POS = giorni_max_posizione
        self.log = None
        self.model = None
        self.valutazione_modello = None
        self.posizioni = None
        self.bilancio = None
        self.esito = None
        
    def avvia_trading(self, verbose=0) -> None:
        self.log = "\033[42m" + self.SIMBOLO + "\033[0m" + "\n"
        ticker = yf.download(self.SIMBOLO, start=self.DATA_INIZIO, end=self.DATA_FINE, progress=False)
        ticker.index = ticker.index.date
        str = f"Calcolo indicatori ticker {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"
        ticker = fx.crea_indicatori(ticker)
        ticker = fx.imposta_target(ticker)
        ticker.dropna(axis=0, inplace=True)

        str = f"Definizione features e target {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"        
        idx, X, Y, _ = fx.to_XY(ticker, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, bilanciamento=0)
        str = f"Previsione {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"        
        f1_score = F1Score(num_classes=1, average='macro', threshold=self.PROBABILITA_PER_ACQUISTO)
        self.model = load_model("model.h5", custom_objects={'Addons>F1Score': f1_score})
        y_pred = self.model.predict(X)
        self.valutazione_modello = self.model.evaluate(X, Y)

        X = X[:, -1, :]

        df_X = ticker.loc[ticker.index.intersection(idx)]
        y_pred = y_pred.reshape(-1,)
        df_y_pred = pd.DataFrame(y_pred, columns=['prob_True'], index=idx)
        self.dati = pd.concat([df_X, df_y_pred], axis=1)

        self.bilancio = self.BILANCIO_INIZIALE
        pos_aperta = False
        tp = sl = n_azioni = prezzo_acquisto = prezzo_tot = 0
        giorni_posizione = 0
        self.posizioni = pd.DataFrame(columns=["Bilancio", "Direzione", "Prezzo_un", "n_azioni", "Prezzo_tot", "TP", "SL", "Open", "High", "Low", "Esito", "P_L"], index=self.dati.index)

        for idx, row in self.dati.iterrows():
            if self.bilancio <= 0:
                break
            if pos_aperta == False:
                if row["prob_True"] > self.PROBABILITA_PER_ACQUISTO and self.bilancio > 0:
                    #COMPRA
                    prezzo_acquisto = row["Open"]
                    n_azioni = self.bilancio // prezzo_acquisto
                    prezzo_tot = n_azioni * prezzo_acquisto  
                    self.bilancio -= prezzo_tot          
                    tp = prezzo_acquisto * (1 + self.TP)
                    sl = prezzo_acquisto * (1 - self.SL)
                    pos_aperta = True
                    giorni_posizione = 0
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "COMPRA", "Prezzo_un": prezzo_acquisto, "n_azioni": n_azioni, "Prezzo_tot": -prezzo_tot, "TP": tp, "SL": sl, "Open": row["Open"], "High": row["High"], "Low": row["Low"],}
            else:
                if giorni_posizione == self.GIORNI_POS:
                    prezzo_tot_vendita = row['Close'] * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    if prezzo_tot_vendita > prezzo_tot:
                        esito = 'VINCITA'
                    else:
                        esito = 'PERDITA'
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": row['Close'], "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": esito, "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                if row["High"] >= tp:
                    prezzo_tot_vendita = tp * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    giorni_posizione = 0
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": tp, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "VINCITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                elif row["Low"] <= sl:
                    prezzo_tot_vendita = sl * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    giorni_posizione = 0
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": sl, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "PERDITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                else:
                    giorni_posizione += 1

        if pos_aperta: 
            self.bilancio += prezzo_tot

        tot_vincite = self.posizioni.loc[self.posizioni["Esito"] == "VINCITA", "Esito"].count()
        tot_perdite = self.posizioni.loc[self.posizioni["Esito"] == "PERDITA", "Esito"].count()
        max_vincita_value = self.posizioni.loc[self.posizioni["Esito"] == "VINCITA", "P_L"].max(skipna=True)
        max_perdita_value = self.posizioni.loc[self.posizioni["Esito"] == "PERDITA", "P_L"].min(skipna=True)
        max_vincita = int(max_vincita_value if max_vincita_value == max_vincita_value else 0)  # max_vincita_value == max_vincita_value è un modo per controllare se non è NaN
        max_perdita = int(max_perdita_value if max_perdita_value == max_perdita_value else 0)
        tot = int(fx.pct_change(self.BILANCIO_INIZIALE, self.bilancio))
        self.esito = (
            f"\nBilancio {self.SIMBOLO} = {round(self.bilancio, 2)}" + "\n" +
            f'Vincite: {tot_vincite}, vincita max: {max_vincita} %' + "\n" + 
            f'Perdite: {tot_perdite}, perdita max: {max_perdita} %' + "\n" +
            f'Totale: {tot} %'
        )
        if verbose == 1: print(self.esito)
        self.log += self.esito + "\n"